# Chat With Avatar About My Experience and Skills

In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

In [2]:
load_dotenv(override=True)

openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")
groq_api_key = os.getenv("GROQ_API_KEY")


In [ ]:
openai = OpenAI()
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [ ]:
reader = PdfReader("../me/Linkedin_Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [ ]:
# print(linkedin)

In [ ]:
with open("../me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [ ]:
with open("../me/current_situation.txt", "r", encoding="utf-8") as f:
    current_situation = f.read()

In [ ]:
name = "Mariusz Bronowicki"

In [ ]:
system_prompt = f"You are acting as {name}. You are answering question on {name}'s website, \
particularly question related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you do not know the answer, say so. \
If you need to check e.g salary expectation question then use tools to see what range for such position is."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## Linkedin Profile:\n{linkedin}\n\n## Current situation:\n{current_situation}\n\n"
system_prompt += f"With this context, please chat with user, always staying in character as {name}."

In [ ]:
system_prompt

In [ ]:
def chat_gpt(message, history):
    messages = [{"role": "user", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat_gemini(message, history):
    history = [{"role": h["role"], "content": h["content"]} for h in history]
    messages = [{"role": "user", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model="gemini-2.0-flash", messages=messages)
    return response.choices[0].message.content

In [ ]:
gr.ChatInterface(chat_gpt, type="messages").launch()

In [ ]:
gr.ChatInterface(chat_gemini, type="messages").launch()

## Ask LLM to evaluate answer from previous model.

All without any Agentic Framework!

In [ ]:
# Create a Pydantic model for the Evaluation
from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str

In [ ]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceeptable. \
You are provided with a conversation btween a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and Linkedin details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## Linkedin Profile{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [ ]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [ ]:
def evaluate(reply, message, history) -> Evaluation:
    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = gemini.chat.completions.parse(model="gemini-2.0-flash", messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [ ]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "What is your current situation?"}]
response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
reply = response.choices[0].message.content

In [ ]:
reply

In [ ]:
evaluate(reply, "What is your current situation?", messages[:1])

In [ ]:
def rerun(reply, message, history,feedback):
    updated_system_prompt = system_prompt + "\n\n## Previous answer rejected\n \
    You just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

In [ ]:
def chat(message, history):
    # if "tell me about yourself" in message:
    #     system = system_prompt + "\n\nEverything in you reply needs to be in pig latin - \
    #         it is mandatory that you response only and entirely in pig latin"
    # else:
    #     system = system_prompt
    system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content

    evaluation = evaluate(reply, message, history)

    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

In [3]:
import pyttsx3

# Initialize the TTS engine
engine = pyttsx3.init()

# Set properties (optional)
engine.setProperty('rate', 150) # Speed of speech (words per minute)
engine.setProperty("volume", 1.0) # Volume (0.0 to 1.0)

# Text to speak
text_to_read = "Hello! I’m Mariusz Bronowicki, a professional committed to delivering high-quality work in my field. \
                I have a diverse background and skill set that allows me to tackle various challenges effectively. \
                If you have any questions about my career, experience, or skills, feel free to ask! I'm here to help."

# Speak the text
engine.say(text_to_read)

# Wait until speaking is finishing
engine.runAndWait()

In [ ]:
from openai import OpenAI

text_to_read = "Hello! I’m Mariusz Bronowicki, a professional committed to delivering high-quality work in my field. \
                I have a diverse background and skill set that allows me to tackle various challenges effectively. \
                If you have any questions about my career, experience, or skills, feel free to ask! I'm here to help."


client = OpenAI()

audio = client.audio.speech.create(
    model="gpt-4o-mini-tts",
    voice="alloy",
    input=text_to_read
)

# Save to file
with open("../me/output.mp3", "wb") as f:
    f.write(audio.read())